# ElasticNet Regression
- Amaç hata kareler toplamını(SSE) minimize eden katsayıları bu katsayılara bir ceza uygulayarak bulmaktır.
- ElasticNet L1 ve L2 yaklaşımlarını birleştirir. Yani Ridge ve Lasso yönetmlerini birleştirir.
- ElasticNet Ridge tarzı cezalandırma, Lasso tarzı ise Değişken seçimi yapıyor ve bu iki işlemi birleştiriyor.
- Diğer doğrusal modellere göre bazı açılardan daha iyi iş yapmış oluyor.


**Gerekli Kütüphaneler**

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn import model_selection

**Veri Seti**

In [15]:
df = pd.read_csv("./Hitters.csv")
# bu csv dosyasının içerisinde eksik gözlemleri(NA) çıkardık.
df = df.dropna()

dms = pd.get_dummies(df[["League","Division","NewLeague"]])

# bağımlı değişken
y = df[["Salary"]]

# burada yazdığımız verileri uçurduk drop ile
X_ = df.drop(["Salary","League","Division","NewLeague"], axis = 1).astype("float64")

# dms ile X_ birleştirip(concat) bağımsız değişken oluşturduk.
X = pd.concat([X_, dms[["League_N","Division_W","NewLeague_N"]]], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.25,
                                                    random_state= 42) 

**Model Oluşturma** 

In [16]:
elasticNet_model = ElasticNet().fit(X_train,y_train)

C:\Users\halil\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8487984.799367279, tolerance: 3898.686956380658
  model = cd_fast.enet_coordinate_descent(


In [18]:
elasticNet_model

ElasticNet()

In [17]:
elasticNet_model.coef_

array([ -1.86256172,   8.70489065,   5.10426375,  -2.89875799,
        -1.28642985,   5.24343682,   6.04480276,  -0.14701495,
        -0.21566628,  -0.7897201 ,   1.80813117,   0.80914508,
        -0.61262382,   0.26816203,   0.27172387,  -0.36530729,
        19.2186222 , -31.16586592,   8.98369938])

In [20]:
elasticNet_model.intercept_

array([-6.4659556])

**Tahmin**

In [25]:
elasticNet_model.predict(X_train)[0:10]

array([325.74706292, 776.06632333, 522.86508419, 107.64091955,
       449.03139566, 997.76095723,  99.78828622, 311.33763086,
       418.50335021, 879.9502608 ])

In [26]:
elasticNet_model.predict(X_test)[0:10]

array([ 577.79111731,  617.33202224, 1031.39113156,  364.95861575,
        489.51894393,  300.74185842,  604.522666  ,  465.34678732,
        901.44473965,  703.20357123])

In [27]:
y_pred = elasticNet_model.predict(X_test)

In [29]:
# RMSE = hata kareler ortalamasının kare kökü
RMSE = np.sqrt(mean_squared_error(y_test,y_pred))
RMSE

357.1676548181244

In [31]:
#r2_score'u
r2_score(y_test,y_pred)

0.41070222469326945

**Model Tuning**

In [37]:
enet_cv_model = ElasticNetCV(cv = 10).fit(X_train,y_train)

C:\Users\halil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [38]:
enet_cv_model_alpha = enet_cv_model.alpha_

In [39]:
enet_cv_model.intercept_

-38.5194055839429

In [40]:
enet_cv_model.coef_

array([ 0.62845434,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.        ,  0.        ,  0.09788752,  0.        ,
        0.27265769,  0.19270075,  0.00758665,  0.3106529 ,  0.        ,
       -0.        ,  0.        , -0.        ,  0.        ])

**Final Modeli**

In [42]:
#Final Modeli
enet_tuned = ElasticNet(enet_cv_model_alpha).fit(X_train,y_train)

In [44]:
y_pred = enet_tuned.predict(X_test)

In [46]:
RMSE = np.sqrt(mean_squared_error(y_test,y_pred))
RMSE

394.15280563218795

In [47]:
?ElasticNet

Init signature:
ElasticNet(
    alpha=1.0,
    *,
    l1_ratio=0.5,
    fit_intercept=True,
    normalize=False,
    precompute=False,
    max_iter=1000,
    copy_X=True,
    tol=0.0001,
    warm_start=False,
    positive=False,
    random_state=None,
    selection='cyclic',
)
Docstring:     
Linear regression with combined L1 and L2 priors as regularizer.

Minimizes the objective function::

        1 / (2 * n_samples) * ||y - Xw||^2_2
        + alpha * l1_ratio * ||w||_1
        + 0.5 * alpha * (1 - l1_ratio) * ||w||^2_2

If you are interested in controlling the L1 and L2 penalty
separately, keep in mind that this is equivalent to::

        a * L1 + b * L2

where::

        alpha = a + b and l1_ratio = a / (a + b)

The parameter l1_ratio corresponds to alpha in the glmnet R package while
alpha corresponds to the lambda parameter in glmnet. Specifically, l1_ratio
= 1 is the lasso penalty. Currently, l1_ratio <= 0.01 is not reliable,
unless you supply your own sequence of alpha.

Read